## Graph DB 연결하기
* Crime Investigation Dataset: https://github.com/neo4j-graph-examples/pole

In [1]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://ddadee7d.databases.neo4j.io"
AUTH = ("neo4j", "dBUuO0ettwv7MK-R1UsGAPLUs7hA22-YltunltI4d18")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [2]:
# 로컬에 있는 .env 를 통해 API key 불러옴 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "neo4j://18.212.35.48:7687",
  auth=basic_auth("neo4j", "swamp-surges-rag"))

In [4]:
from neo4j_graphrag.llm.openai_llm import OpenAILLM

llm = OpenAILLM(model_name="gpt-4o")

In [5]:
cypher_query = '''
MATCH (l:Location {address:$address})<-[r:OCCURRED_AT]-(c:Crime)
RETURN c.date as crimeDate
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      address="Piccadilly").data())
  for record in results:
    print(record['crimeDate'])

driver.close()

C:\Users\User\AppData\Local\Temp\ipykernel_74512\2601851126.py:7: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


23/08/2017
18/08/2017
18/08/2017
3/08/2017
27/08/2017
15/08/2017
12/08/2017
11/08/2017
5/08/2017
10/08/2017
6/08/2017
1/08/2017
31/08/2017
21/08/2017
1/08/2017
20/08/2017
30/08/2017
19/08/2017
26/08/2017
7/08/2017
30/08/2017
10/08/2017
23/08/2017
30/08/2017
1/08/2017
1/08/2017
16/08/2017
30/08/2017
19/08/2017
6/08/2017
11/08/2017
25/08/2017
1/08/2017
5/08/2017
27/08/2017
22/08/2017
3/08/2017
5/08/2017
11/08/2017
26/08/2017
2/08/2017
31/08/2017
1/08/2017
25/08/2017
9/08/2017
14/08/2017
13/08/2017
16/08/2017
25/08/2017
9/08/2017
21/08/2017
26/08/2017
28/08/2017
18/08/2017
19/08/2017
11/08/2017
26/08/2017
3/08/2017
16/08/2017
23/08/2017
29/08/2017
19/08/2017
26/08/2017
29/08/2017
13/08/2017
17/08/2017
19/08/2017
26/08/2017
7/08/2017
11/08/2017
17/08/2017
26/08/2017
22/08/2017
4/08/2017
6/08/2017
25/08/2017
25/08/2017
22/08/2017
23/08/2017
2/08/2017
25/08/2017
26/08/2017
30/08/2017
6/08/2017
7/08/2017
26/08/2017
4/08/2017
11/08/2017
3/08/2017
10/08/2017
12/08/2017
20/08/2017
21/08/2017
17/

## 1. Graph 기반 RAG 구현하기: Text2Cyper

### 1) POLE(범죄수사데이터셋) 데이터 그래프 이해하기

### 2) Text2Cypher Retriever 사용하기

#### 2-1) DB 스키마 작성하기
- 리트리버를 사용하기 위해서는 LLM을 쓸 것임 
- LLM이 DB 스키마를 알고있어야지, 쿼리문을 잘 생성해낼 수 있음 
- 즉, 리트리버가 잘 작동하게 하기 위해서는 DB 스키마를 명시해줘야함 

#### 2-1-1) 방법1: 노드, 관계 프로퍼티, 관계방향성을 나타내는 스키마 작성 

In [6]:
from collections import defaultdict

def get_schema():
    schema = ""
    with driver.session() as session:
        # 모든 노드 라벨과 속성 추출
        node_schema = session.run("""
        CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName, propertyTypes
        RETURN nodeType, propertyName, propertyTypes
        """)

        nodes = defaultdict(dict)
        for record in node_schema:
            label = record["nodeType"].replace(":", "")
            prop = record["propertyName"]
            types = record["propertyTypes"]
            nodes[label][prop] = types[0] if types else "UNKNOWN"

        # 모든 관계 타입과 속성 추출
        rel_schema = session.run("""
        CALL db.schema.relTypeProperties() YIELD relType, propertyName, propertyTypes
        RETURN relType, propertyName, propertyTypes
        """)

        relationships = defaultdict(dict)
        for record in rel_schema:
            rel = record["relType"]
            prop = record["propertyName"]
            types = record["propertyTypes"]
            relationships[rel][prop] = types[0] if types else "UNKNOWN"

        # 관계 방향 및 타입 추출
        rel_types = session.run("""
        MATCH (a)-[r]->(b)
        RETURN DISTINCT labels(a) AS from_labels, type(r) AS rel_type, labels(b) AS to_labels
        """)

        rel_directions = set()
        for record in rel_types:
            from_label = f":{record['from_labels'][0]}"
            to_label = f":{record['to_labels'][0]}"
            rel_type = record['rel_type']
            rel_directions.add(f"({from_label})-[:{rel_type}]->({to_label})")

    # 출력
    schema += "\nNode properties:\n"
    for label, props in nodes.items():
        prop_str = ", ".join(f"{k}: {v}" for k, v in props.items())
        schema += f"{label} {{{prop_str}}}\n"

    schema += "\nRelationship properties:\n"
    for rel, props in relationships.items():
        prop_str = ", ".join(f"{k}: {v}" for k, v in props.items())
        schema += f"{rel} {{{prop_str}}}\n"

    schema += "\nThe relationships:\n"
    for rel in sorted(rel_directions):
        schema += f"{rel}\n"
    return schema

schema = get_schema()

C:\Users\User\AppData\Local\Temp\ipykernel_74512\2814877246.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Procedure.ProcedureWarning} {category: GENERIC} {title: The query used a procedure that generated a warning.} {description: The query used a procedure that generated a warning. (The field `propertyTypes` will change output format in the next major version.)} {position: line: 2, column: 9, offset: 9} for query: '\n        CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName, propertyTypes\n        RETURN nodeType, propertyName, propertyTypes\n        '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Procedure.ProcedureWarning} {category: GENERIC} {title: The query used a procedure that generated a warning.} {description: 

In [7]:
print(schema)


Node properties:
`Location` {address: String, postcode: String, longitude: Double, latitude: Double}
`Phone` {phoneNo: String}
`Email` {email_address: String}
`Officer` {name: String, surname: String, badge_no: String, rank: String}
`PostCode` {code: String}
`Area` {areaCode: String}
`PhoneCall` {call_date: String, call_type: String, call_duration: String, call_time: String}
`Crime` {id: String, date: String, type: String, last_outcome: String, charge: String, note: String}
`Object` {id: String, type: String, description: String}
`Vehicle` {make: String, model: String, year: String, reg: String}
`Person` {nhs_no: String, name: String, surname: String, age: String}

Relationship properties:
:`CURRENT_ADDRESS` {None: UNKNOWN}
:`HAS_PHONE` {None: UNKNOWN}
:`HAS_EMAIL` {None: UNKNOWN}
:`HAS_POSTCODE` {None: UNKNOWN}
:`POSTCODE_IN_AREA` {None: UNKNOWN}
:`LOCATION_IN_AREA` {None: UNKNOWN}
:`KNOWS_SN` {None: UNKNOWN}
:`KNOWS` {None: UNKNOWN}
:`CALLER` {None: UNKNOWN}
:`CALLED` {None: UNKNOWN

### 2-1-2) 방법2:JSON 으로 추출 
* 장점이 description을 수동으로 입력해서, LLM 이 더 잘 알아들을 수 있게 처리 가능

## 2-2) Text2Cyper Retriever 로 검색하기

In [9]:
from neo4j import GraphDatabase
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.llm import OpenAILLM

neo4j_schema = get_schema()

examples = [
    """
    USER INPUT: Piccadilly에서 발생한 범죄 건수는?
    QUERY: MATCH (c:Crime)-[:OCCURRED_AT]->(l:Location {address: 'Piccadilly'})
    RETURN count(c) AS crime_count

    USER INPUT: M22 8ER 우편번호 지역의 범죄 유형 분포는?
    QUERY: MATCH (pc:PostCode {code: "M22 8ER"})<-[:HAS_POSTCODE]-(l:Location)<-[:OCCURRED_AT]-(c:Crime)
    RETURN c.type, count(*) AS count
    ORDER BY count DESC

    USER INPUT: Larive 경관(badge no 26-5234182)이 수사 중인 사건은 무엇인가요?
    QUERY: MATCH (c:Crime {last_outcome: 'Under investigation'})-[i:INVESTIGATED_BY]->(o:Officer {badge_no: '26-5234182', surname: 'Larive'})
    return *
    """
]

retriever = Text2CypherRetriever(
    driver = driver, 
    llm = llm, 
    neo4j_schema = neo4j_schema,
    examples = examples
)

C:\Users\User\AppData\Local\Temp\ipykernel_74512\2814877246.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Procedure.ProcedureWarning} {category: GENERIC} {title: The query used a procedure that generated a warning.} {description: The query used a procedure that generated a warning. (The field `propertyTypes` will change output format in the next major version.)} {position: line: 2, column: 9, offset: 9} for query: '\n        CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName, propertyTypes\n        RETURN nodeType, propertyName, propertyTypes\n        '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Procedure.ProcedureWarning} {category: GENERIC} {title: The query used a procedure that generated a warning.} {description: 

In [59]:
query_text = "현재 수사중인 범죄 건수가 많이 배정된 담당자는 누구야? 이 담당자가 담당하는 정보들에 대하여 자세히 설명해줘"
result = retriever.search(query_text = query_text)

In [60]:
result.items

[RetrieverResultItem(content="<Record o.name='Hernando' o.surname='Macias' o.badge_no='55-5623906' o.rank='Sergeant' c.id='308f1943cc94b771b4d1a2a24b2671d9ffba62eb0ec53f14fda5f01c87095433' c.date='28/08/2017' c.type='Public order' c.last_outcome='Under investigation' c.charge=None c.note=None>", metadata=None),
 RetrieverResultItem(content="<Record o.name='Hernando' o.surname='Macias' o.badge_no='55-5623906' o.rank='Sergeant' c.id='b76f501b4e10c02708ff2fd7ff49d079378efe95a108eeeff9b0cb81996bd5e8' c.date='12/08/2017' c.type='Violence and sexual offences' c.last_outcome='Under investigation' c.charge=None c.note=None>", metadata=None),
 RetrieverResultItem(content="<Record o.name='Hernando' o.surname='Macias' o.badge_no='55-5623906' o.rank='Sergeant' c.id='d894d916e6eb9ca5bc0dee9c5b52adf208eebf80be2a23865e36c56180971a44' c.date='24/08/2017' c.type='Violence and sexual offences' c.last_outcome='Under investigation' c.charge=None c.note=None>", metadata=None),
 RetrieverResultItem(content=

In [61]:
print(result.metadata["cypher"])

MATCH (c:Crime {last_outcome: 'Under investigation'})-[:INVESTIGATED_BY]->(o:Officer)
WITH o, count(c) AS crime_count
ORDER BY crime_count DESC
LIMIT 1
MATCH (c:Crime {last_outcome: 'Under investigation'})-[:INVESTIGATED_BY]->(o)
RETURN o.name, o.surname, o.badge_no, o.rank, c.id, c.date, c.type, c.last_outcome, c.charge, c.note


### 3) GraphRAG 파이프라인 모듈 사용하기
* 리트리버를 통해 찾은 정보들을 llm이 한번 더 처리해서 답변해주는 부분 

In [86]:
from neo4j_graphrag.llm.openai_llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG

llm = OpenAILLM(model_name = "gpt-4o")
graph_rag = GraphRAG(retriever, llm)

In [91]:
query = "범죄자는 아니지만, 범죄자를 많이 알고 있는사람에 대하여 알려줘."
response = graph_rag.search(query_text=query, return_context=True)

In [92]:
print("## 생성 답변 ##\n" + response.answer)
print("\n## Cypher ##\n" + response.retriever_result.metadata["cypher"])
print("\n## 검색 결과 ##")
for i in response.retriever_result.items: print(i.content)

## 생성 답변 ##
범죄자는 아니지만, 범죄자를 많이 알고 있는 사람은 Anne Freeman로, 그녀는 7명의 범죄자를 알고 있습니다.

## Cypher ##
MATCH (p:Person)-[:KNOWS]->(c:Person)-[:PARTY_TO]->(cr:Crime)
WHERE NOT (p)-[:PARTY_TO]->(:Crime)
RETURN p.name, p.surname, count(DISTINCT c) AS known_criminal_count
ORDER BY known_criminal_count DESC

## 검색 결과 ##
<Record p.name='Anne' p.surname='Freeman' known_criminal_count=7>
<Record p.name='Bonnie' p.surname='Gilbert' known_criminal_count=6>
<Record p.name='Ashley' p.surname='Robertson' known_criminal_count=3>
<Record p.name='Walter' p.surname='James' known_criminal_count=2>
<Record p.name='Pamela' p.surname='Gibson' known_criminal_count=1>
<Record p.name='Evelyn' p.surname='Wood' known_criminal_count=1>
<Record p.name='Melissa' p.surname='Mills' known_criminal_count=1>
<Record p.name='Arthur' p.surname='Willis' known_criminal_count=1>
<Record p.name='Catherine' p.surname='White' known_criminal_count=1>
<Record p.name='Louis' p.surname='Richards' known_criminal_count=1>
<Record p.name='Matth